In [1]:
import pandas as pd
import datetime  
from random import randint

In [2]:
def insert_activity(c_id,activity,time):
    data.loc[data.shape[0]] = [c_id,activity,time]
    return 0

In [3]:
def get_info_from_client():
    return 'Client in case'

def client_accept():
    return 'Client accepted'

def client_reject():
    return 'Client rejected'

def check_cred_story():
    return 'Cred story in check'

def bad_cred_story():
    return 'Client rejected by story'

def good_cred_story():
    return 'Client accepted by story'

def under_check():
    return 'Underraiter check'

def under_accept():
    return 'Client accepted by underwriter'

def under_reject():
    return 'Client rejected by underwriter'

def give_answer():
    return 'Answer done'

def something_may_be_wrong(stage):
    return 'Something wrong on stage ' + stage


In [4]:
def generate_problem(c_id,activity,time):
    if randint(0,100)>98:
        insert_activity(c_id,activity,time)
    return 0

In [5]:
def get_process_log_by_client_with_errors(c_id):
    
    state=1
    time=datetime.datetime.now()
    
    insert_activity(c_id,get_info_from_client(),time)
    generate_problem(c_id,something_may_be_wrong('GET_INFO'),time)
    
    time+=datetime.timedelta(days=1)
    insert_activity(c_id,check_cred_story(),time)
    
    
    generate_problem(c_id,something_may_be_wrong('STORY_CHECK'),time)
    time+=datetime.timedelta(days=1)
    if randint(0,100)<92:
        insert_activity(c_id,good_cred_story(),time)
    else:
        insert_activity(c_id,bad_cred_story(),time)
        state = 0
        
        
    time+=datetime.timedelta(days=1)
    if state == 1:
        generate_problem(c_id,something_may_be_wrong('UNDER_CHECK'),time)
        insert_activity(c_id,under_check(),time)
        if randint(0,100)<80:
            insert_activity(c_id,under_accept(),time)
        else:
            insert_activity(c_id,under_reject(),time)
            state = 0
            
            
    generate_problem(c_id,something_may_be_wrong('RESOLUTION'),time)
    time+=datetime.timedelta(days=1)
    if state == 1:
        insert_activity(c_id,client_accept(),time)
    else:
        insert_activity(c_id,client_reject(),time)
        
        
    generate_problem(c_id,something_may_be_wrong('ANSWER'),time)
    time+=datetime.timedelta(days=1)
    insert_activity(c_id,give_answer(),time)
    
    return 0

In [6]:
def get_process_log_by_client(c_id):
    
    state=1
    time=datetime.datetime.now()
    
    insert_activity(c_id,get_info_from_client(),time)
    
    time+=datetime.timedelta(days=1)
    insert_activity(c_id,check_cred_story(),time)
    
    time+=datetime.timedelta(days=1)
    if randint(0,100)<92:
        insert_activity(c_id,good_cred_story(),time)
    else:
        insert_activity(c_id,bad_cred_story(),time)
        state = 0
    
    time+=datetime.timedelta(days=1)
    if state == 1:
        insert_activity(c_id,under_check(),time)
        if randint(0,100)<80:
            insert_activity(c_id,under_accept(),time)
        else:
            insert_activity(c_id,under_reject(),time)
            state = 0
    
    time+=datetime.timedelta(days=1)
    if state == 1:
        insert_activity(c_id,client_accept(),time)
    else:
        insert_activity(c_id,client_reject(),time)
    
    time+=datetime.timedelta(days=1)
    insert_activity(c_id,give_answer(),time)
    
    return 0

In [7]:
data = pd.DataFrame(columns=['id','concept:name','time:timestamp'])
for c_id in range(1,1001):
    get_process_log_by_client_with_errors(c_id)

In [8]:
data.to_csv('log.csv',index=False)

In [9]:
from pm4py.algo.discovery.alpha import factory as miner
from pm4py.objects.log.importer.csv import factory as imp
from pm4py.objects.conversion.log import factory as conv
from pm4py.util import constants
from pm4py.statistics.traces.log import case_statistics

In [10]:
stream = imp.apply('log.csv')

In [11]:
log = conv.apply(stream,parameters={constants.PARAMETER_CONSTANT_CASEID_KEY:"id",
                                    constants.PARAMETER_CONSTANT_ACTIVITY_KEY:"concept:name",
                                   constants.PARAMETER_CONSTANT_TIMESTAMP_KEY:"time",
                                   })


In [12]:
net, initial_marking, final_marking = miner.apply(log)

In [ ]:
from pm4py.algo.discovery.dfg import factory as dfg_f
from pm4py.visualization.dfg import factory as dfg_vis

dfg = dfg_f.apply(log)
gviz = dfg_vis.apply(dfg,log,variant='frequency')
pet.save(gviz,'viz_dfg.png')

In [13]:
from pm4py.visualization.petrinet import factory as vis


gviz = vis.apply(net,initial_marking,final_marking)
vis.save(gviz,'viz_descr.png')

In [19]:
case_statistics.get_events(log,case_id=357)

[{'id': 357,
  'concept:name': 'Client in case',
  'time:timestamp': Timestamp('2019-09-18 13:06:08.795016+0000', tz='UTC')},
 {'id': 357,
  'concept:name': 'Cred story in check',
  'time:timestamp': Timestamp('2019-09-19 13:06:08.795016+0000', tz='UTC')},
 {'id': 357,
  'concept:name': 'Client rejected by story',
  'time:timestamp': Timestamp('2019-09-20 13:06:08.795016+0000', tz='UTC')},
 {'id': 357,
  'concept:name': 'Client rejected',
  'time:timestamp': Timestamp('2019-09-22 13:06:08.795016+0000', tz='UTC')},
 {'id': 357,
  'concept:name': 'Answer done',
  'time:timestamp': Timestamp('2019-09-23 13:06:08.795016+0000', tz='UTC')}]

In [ ]:
dfg = dfg_f.apply(log,variant='performance')
gviz = dfg_vis.apply(dfg,log,variant='performance')
dfg_vis.save(gviz,'viz_perf.png')

In [16]:
var_with_count = case_statistics.get_variant_statistics(log)

In [17]:
var_with_count

[{'variant': 'Client in case,Cred story in check,Client accepted by story,Underraiter check,Client accepted by underwriter,Client accepted,Answer done',
  'count': 670},
 {'variant': 'Client in case,Cred story in check,Client accepted by story,Underraiter check,Client rejected by underwriter,Client rejected,Answer done',
  'count': 165},
 {'variant': 'Client in case,Cred story in check,Client rejected by story,Client rejected,Answer done',
  'count': 73},
 {'variant': 'Client in case,Cred story in check,Client accepted by story,Underraiter check,Client accepted by underwriter,Client accepted,Something wrong on stage ANSWER,Answer done',
  'count': 17},
 {'variant': 'Client in case,Cred story in check,Client accepted by story,Something wrong on stage UNDER_CHECK,Underraiter check,Client accepted by underwriter,Client accepted,Answer done',
  'count': 16},
 {'variant': 'Client in case,Something wrong on stage GET_INFO,Cred story in check,Client accepted by story,Underraiter check,Client 

In [26]:
log[0].attributes

{'concept:name': 1}